In [ ]:
import csv
import json
import pathlib
from collections import defaultdict
csv.field_size_limit(sys.maxsize)

In [ ]:
package_filenames = list(pathlib.Path("./packages/").glob("*.txt"))

In [ ]:
filename = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/11/image_ACR_20211115.tsv"

series = defaultdict(list)

with open(filename) as manifest_file:
    reader = csv.DictReader(manifest_file, delimiter='\t')
    for row in reader:
        series_id = row["series.submitter_id"]
        filename = row["file_name"]

        md5sum = row["mdsum"]
        filesize = int(row["file_size"])

        package_contents = {"hashes": {"md5sum": md5sum}, "size": filesize}

        series[(series_id, filename)].append(package_contents)

In [ ]:
all_packages = []

for package_filename in package_filenames:
    with open(package_filename) as package_file:
        tsv_reader = csv.DictReader(package_file, delimiter="\t", quotechar='', quoting=csv.QUOTE_NONE)
        for record in tsv_reader:
            # record["package_contents"] = json.dumps(json.loads(record["package_contents"].replace("'", "\"")))
            record["package_contents"] = json.loads(record["package_contents"].replace("'", "\""))
            all_packages.append(record)

In [ ]:
all_packages_filled = []

for package in all_packages:
    series_id = package["file_name"].split("/")[1].rsplit(".", 1)[0]

    for pc in package["package_contents"]:
        fname = pc["file_name"].split("/")[1]
        pc.update(series[(series_id, fname)][0])
    
    package["package_contents"] = json.dumps(package["package_contents"])
    all_packages_filled.append(package)

In [ ]:
with open("all_packages.tsv", "w") as all_packages_file:
    fieldnames = ['record_type', 'guid', 'md5', 'size', 'authz', 'url', 'file_name', 'package_contents']
    tsv_writer = csv.DictWriter(all_packages_file, delimiter="\t", quotechar='', quoting=csv.QUOTE_NONE, fieldnames=fieldnames)
    tsv_writer.writeheader()

    for package in all_packages_filled:
        tsv_writer.writerow(package)